In [1]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 69.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=569ac2b79ba2b8d2afd7b1da8cdfc6962d3af05e9a0b98e717326c2a1e0963dd
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 77 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=178684386214d933544e9dcf0bf753a644e2805cbbb0719204e90577d31dd4d8
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark


In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('/content/drive/MyDrive/Spark with Python/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
for item in data.head(2)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=[ 'Avg Session Length','Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select('features','Yearly Amount Spent' )

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean|  497.2480393740644|
| stddev|  81.01790454242655|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
test_data.describe().show()


+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean| 504.13470232248585|
| stddev|  75.23320831525912|
|    min|   266.086340948469|
|    max|  662.9610878058954|
+-------+-------------------+



In [24]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [25]:
lr_model = lr.fit(train_data)

In [26]:
test_results = lr_model.evaluate(test_data)

In [27]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-13.009093221241187|
| -18.26318251260716|
|  -7.42670769768921|
|-14.146731156365774|
|  8.316707660600969|
| -4.630205318555909|
| 17.074428959865884|
| 2.5838690144277052|
| -7.799756972349087|
|-19.128177296073318|
|-15.793997416679304|
| -8.702963398453107|
| -7.369882525306991|
| 2.1515968602230373|
|  6.843911480922941|
|   7.57618023303246|
| -5.920847293759948|
|-0.6767428029336884|
| 2.4091424787865776|
|-12.216437851907642|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

10.83253153631861

In [29]:
test_results.r2

0.9791288802735034

In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8162006488763...|
|[31.0613251567161...|
|[31.0662181616375...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3123495994443...|
|[31.3584771924370...|
|[31.5147378578019...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.8209982016720...|
|[31.9549038566348...|
|[31.9673209478824...|
|[32.0047530203648...|
|[32.0123007682454...|
|[32.0637746203136...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = lr_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.9379630244348|
|[30.8162006488763...|284.34952346107616|
|[31.0613251567161...| 494.9821657555908|
|[31.0662181616375...| 463.0800243640401|
|[31.1695067987115...|419.03982314169184|
|[31.2681042107507...|428.10073849237983|
|[31.3123495994443...|446.51698906807474|
|[31.3584771924370...| 492.5920814350477|
|[31.5147378578019...| 497.6122449688105|
|[31.5702008293202...| 565.0736694374782|
|[31.5741380228732...| 560.2032695772662|
|[31.7207699002873...|  547.477896876476|
|[31.7242025238451...| 510.7577698132675|
|[31.7366356860502...| 494.7818493953089|
|[31.8209982016720...| 417.8313695322904|
|[31.9549038566348...| 432.4216997068945|
|[31.9673209478824...| 451.6706885334122|
|[32.0047530203648...| 464.4227239235631|
|[32.0123007682454...|490.53591058717166|
|[32.0637746203136...|390.54734475871146|
+--------------------+------------